# Exercise 2: Logistic Regression

The [Wisconsin Breast Cancer dataset](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) contains features computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. After segmenting cell nuclei present in the image, ten nuclear features were calculated for each cell. These features are modeled such that higher values are typically associated with malignancy. The mean value, worst (mean of the three largest values), and standard error of each feature were computed for each image, resulting in a total of 30 features for each case in the study:

1. radius (mean of distances from center to points on the perimeter)
2. texture (standard deviation of gray-scale values)
3. perimeter
4. area
5. smoothness (local variation in radius lengths)
6. compactness (perimeter^2 / area - 1.0)
7. concavity (severity of concave portions of the contour)
8. concave points (number of concave portions of the contour)
9. symmetry 
10. fractal dimension ("coastline approximation" - 1)

The objective is to determine how points can best be separated into *benign* and *malignant* sets in the case of diagnosis, and into *recurring* and *nonrecurring* sets in the case of prognosis.

## Weighted Least Squares

Weighted Least Squares (WLS) is an extension to least squares, where each sample
is associated with a weight $w_i$. Accordingly, the weighted least squares estimate
is defined as
$$
\hat{\beta} = \left( \mathbf{X}^\top \mathbf{W} \mathbf{X} \right)^{-1} \mathbf{X}^\top \mathbf{W}^\top \mathbf{y} ,
$$
where $\mathbf{W}$ is a diagonal matrix containing the weights ($W_{ii} = w_i$).

## Iteratively Reweighted Least Squares (IRLS)

The maximum likelihood estimate for logistic regression is defined as
$$
\hat{\beta} = \arg \max_{\beta} \sum_{i = 1}^n y_i \log(\pi_i) + (1 - y_i) \log(1 - \pi_i) ,
$$
where $\pi_i$ denotes the probability $P(y_i = 1 \mid \mathbf{x}_i)$ defined as
$$
 P(y_i = 1 \mid \mathbf{x}_i) = \frac{\exp \left( \beta_0 + \sum_{j=1}^m \beta_j x_{ij} \right)}
{1 + \exp \left( \beta_0 + \sum_{j=1}^m \beta_j x_{ij} \right)}.
$$

In comparison to least squares, there is no closed form solution to this problem,
therefore one has to find estimates iteratively. A common approach is to
iteratively compute a weighted least squares fit until converge, called
**iteratively reweighted least squares** (IRLS).

The weights $w_i$ of the diagonal matrix $\mathbf{W}$ are defined as $w_i = \pi_i (1 - \pi_i)$ and
the **working response** $\mathbf{z} \in \mathbb{R}^n$ as
$$
\mathbf{z} = \mathbf{X} \beta_\text{old} +\mathbf{W}^{-1} (\mathbf{y} - \mathbf{p}) ,
$$
where the vector $\mathbf{p} \in \mathbb{R}^n$ contains the fitted probabilities
$\pi_i$. This leads to the following weighted least squares problem:
$$
\beta_\text{new} = \left( \mathbf{X}^\top \mathbf{W} \mathbf{X} \right)^{-1}
  \mathbf{X}^\top \mathbf{W} \mathbf{z} ,
$$
where the right side of the equation is evaluated at $\beta_\text{old}$.

These equations get solved repeatedly, since at each iteration the vector
$\mathbf{p}$ changes, and hence does $\mathbf{W}$ and $\mathbf{z}$.
Typically, the convergence criteria is based on the **deviance** ($-2 \cdot \text{log-liklihood})$
and is defined as
$$
  \frac{\lvert \mathrm{deviance}({\beta}_\text{new}) - \mathrm{deviance}({\beta}_\text{old}) \rvert}
  {\lvert \mathrm{deviance}({\beta}_\text{old}) \rvert + 0.1} < 10^{-8}
$$

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def image_features():
    names = ['radius', 'texture', 'perimeter', 'area', 'smoothness', 'compactness',
             'concavity', 'concave points', 'symmetry', 'fractal dimension']

    columns = ["mean_{}".format(v) for v in names]
    columns.extend(["se_{}".format(v) for v in names])
    columns.extend(["worst_{}".format(v) for v in names])
    return columns


def load_wisconsin_breast_cancer_diagnosis(local_file='../datasets/wdbc.data'):
    columns = ['ID', 'diagnosis']
    columns.extend(image_features())

    data = pd.read_csv(Path(local_file), index_col=0, header=None,
                       names=columns, dtype={'diagnosis': 'category'})
    y = data.diagnosis.cat.rename_categories(['benign', 'malignant'])
    X = data.drop('diagnosis', axis=1).astype(float)
    return X, y


def load_wisconsin_breast_cancer_prognosis(local_file='../datasets/wpbc.data'):
    columns = ['ID', 'outcome', 'time']
    outcomes = columns[1:]
    extra_vars = ['tumor_size', 'lymph_node_status']
    columns.extend(image_features())
    columns.extend(extra_vars)

    data = pd.read_csv(Path(local_file), index_col=0, header=None,
                       names=columns, dtype={'outcome': 'category'},
                       na_values=['?'])
    y = data.loc[:, outcomes]
    y.outcome.cat.rename_categories(['nonrecurring', 'recurring'], inplace=True)
    X = data.drop(outcomes, axis=1).astype(float)
    return X, y 

## Task 1: Benign vs Malignant

First, we want to classify individual cell nuclei as *benign* or *malignant* based on the 30 features derived from the images.

In [ ]:
X_diagnosis, y_diagnosis = load_wisconsin_breast_cancer_diagnosis()

print(X_diagnosis.shape)
print(y_diagnosis.value_counts())

## Task 2: Recurrence

Next, we want to create a model to predict *recurrence before 24 months* vs. *nonrecurrence beyond 24 months*. The data has two additional features:

1. Tumor size (diameter of the excised tumor in centimeters)
2. Lymph node status (number of positive axillary lymph nodes observed at time of surgery).

**Note:** Lymph node status is missing in 4 cases.

In [ ]:
X_prognosis, y_prognosis = load_wisconsin_breast_cancer_prognosis()

print(X_prognosis.shape)
print(y_prognosis.outcome.value_counts())

plt.hist([
    y_prognosis.time[y_prognosis.outcome == 'recurring'],
    y_prognosis.time[y_prognosis.outcome == 'nonrecurring'],
], label=['recurring', 'nonrecurring'])
plt.ylabel('Frequency')
plt.xlabel('Time')
plt.legend()